se cargan e instalan las librerias necesarias, ademas se inicia el selenium webdriver

In [1]:
pip install webdriver-manager


Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install selenium 

  Attempting uninstall: selenium
    Found existing installation: selenium 4.2.0
    Uninstalling selenium-4.2.0:
      Successfully uninstalled selenium-4.2.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
from selenium import webdriver
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
driver = webdriver.Chrome(ChromeDriverManager().install())
import time
import getpass
from bs4 import BeautifulSoup
#usuario= input("ingrese el usuario para navegar en uss ")
#contraseña =  getpass.getpass("ingrese la contraseña ")
def if_integer(string):
    try: 
        int(string)
        return True
    except ValueError:
        return False

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\rober\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache
C:\Users\rober\AppData\Local\Temp/ipykernel_15648/2412363361.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


se carga la pagina y se realiza el login automatico para iniciar el scraping, la pagina inicial se puede modificar cambiando el link dentro de drive.get

In [4]:
#se cargan la pagina de web of science desde selenium
driver.get("http://www.webofscience.com.bdigitaluss.remotexs.co/wos/woscc/summary/3a69e474-ad67-429a-baf5-12e0c77626b7-4057bdc8/relevance/1")
time.sleep(3)

driver.find_element(By.XPATH,"/html/body/div[4]/div/div/form/div/a").click()
time.sleep(3)
mBox = driver.find_element(By.XPATH,'//*[@id="i0116"]')
mBox.send_keys(usuario)
time.sleep(2)
driver.find_element(By.XPATH,'//*[@id="idSIButton9"]').click()
time.sleep(3)
mBox= driver.find_element(By.XPATH,'//*[@id="i0118"]')
mBox.send_keys(contraseña)
time.sleep(2)
driver.find_element(By.XPATH,'//*[@id="idSIButton9"]').click()
time.sleep(3)
driver.find_element(By.XPATH,'//*[@id="idBtn_Back"]').click()
time.sleep(5)
driver.refresh()


se extraen las paginas una a una copiando el atributo html mediante page_source y este se guarda en una lista de paginas, el numero de paginas se determina mediante total pero se puede cambiar dentro del for in range(total) a for in range(n) con n siendo el numero de paginas a extraer

In [11]:
total =int(driver.find_element(By.XPATH,'/html/body/app-wos/div/div/main/div/div[2]/app-input-route/app-base-summary-component/div/div[2]/app-page-controls[2]/div/form/div/span').text.replace(",",""))
datos =[]
driver.maximize_window()
htmlelement= driver.find_element(By.TAG_NAME,'html')
for i in range(total):
    htmlelement.send_keys(Keys.END)
    time.sleep(1)
    htmlelement.send_keys(Keys.END)
    time.sleep(1)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight/2)")
    time.sleep(1)
    htmlelement.send_keys(Keys.END)
    time.sleep(1)
    htmlelement.send_keys(Keys.HOME)

    d2 = driver.page_source
    datos.append(d2)
    time.sleep(1)
    driver.find_element(By.XPATH,"/html/body/app-wos/div/div/main/div/div[2]/app-input-route/app-base-summary-component/div/div[2]/app-page-controls[1]/div/form/div/button[2]").click()
    time.sleep(1)
        


In [12]:
print(len(datos))

5


se extraen los titulos de cada pagina almacenada en datos y se guardan en una lista data

In [13]:
data= []
for i in range(len(datos)):
    soup =  BeautifulSoup(datos[i])
    for link in soup.find_all('a', attrs={'data-ta': 'summary-record-title-link'}):
        data.append(link.getText())

In [14]:
print(len(data))

250


se extraen las citas de cada pagina almacenada en datos y se guardan en una lista data2, luego esas son limpiadas de espacios y de las palabras citation y citations

In [15]:
data2= []
for i in range(len(datos)):
    soup =  BeautifulSoup(datos[i])
    for link in soup.find_all(attrs={'class':'citations ng-star-inserted'}):
        data2.append(link.getText())

In [16]:
for j in range(len(data2)):
    msg = data2[j]
    removeSpaces = re.sub('\\s+', '',msg)
    data2[j] = removeSpaces

In [17]:
print(len(data2))

165


In [18]:
for i in range(len(data2)):
    data2[i] = data2[i].replace("Citations", "")
    data2[i] = data2[i].replace("Citation", "")

se obtiene el numero de articulos maximos en la pagina, luego se extraen los autores de cada articulo, estos se guardan en la lista data3

In [20]:
tamaño =int(driver.find_element(By.XPATH,'/html/body/app-wos/div/div/main/div/div[2]/app-input-route/app-base-summary-component/div/div[2]/app-page-controls[2]/div/wos-select/button/span[2]').text)
data3= []
for i in range(len(datos)):
    soup =  BeautifulSoup(datos[i])
    for k in range(tamaño):
        for link in soup.find_all(attrs={'data-ta': 'SumAuthTa-' + str(k) + '-MainDiv-author-en'}):
            data3.append(link.getText())

In [21]:
print(len(data3))

250


se obtienen las fechas de publicacion de cada articulo y se guardan en una lista data4, en este caso se creo una lista helpdata4 la cual contiene las fechas que no estan en el formato de las fechas normales, amas se limpian y guardan como fecha normal en sus listas correspondiente

In [22]:
data4= []
for i in range(len(datos)):
    soup =  BeautifulSoup(datos[i])
    for link in soup.find_all(attrs={'data-ta':'summary-record-pubdate'}):
        data4.append(link.getText())

In [23]:
for i in range(len(data4)):
    data4[i] = data4[i].replace("|","")

In [24]:
helpData4= []
for i in range(len(datos)):
    soup =  BeautifulSoup(datos[i])
    for link in soup.find_all(attrs={'class':'source-info-piece ng-star-inserted'}):
        helpData4.append(link.getText())

In [25]:
for i in range(len(helpData4)):
    helpData4[i] = helpData4[i].replace("|","")

se obtienen los titulos de las revistas y se guardan eun una lista data5, en este caso se creo una lista helpdata5 que guarda los titulos de revistas que no contienen un link ya que estos poseen un formato diferente en la pagina

In [26]:
helpData5 = []
for i in range(len(datos)):
    soup =  BeautifulSoup(datos[i])
    for link in soup.find_all(attrs={'class':'font-size-14 summary-source-title ng-star-inserted'}):
        helpData5.append(link.getText())

In [27]:
print(len(helpData5))

119


In [28]:
data5= []
for i in range(len(datos)):
    soup =  BeautifulSoup(datos[i])
    for link in soup.find_all(attrs={'class':'mat-tooltip-trigger font-size-14 borderLess-button thin-focus summary-source-title-link ng-star-inserted'}):
        data5.append(link.getText())

In [29]:
print(len(data5))

131


se extraen las referencias y se guardan en una lista data6, luego se limpian de las palabras references y reference

In [30]:
data6= []
for i in range(len(datos)):
    soup =  BeautifulSoup(datos[i])
    for link in soup.find_all(attrs={'class':'link-container ref-count ng-star-inserted'}):
        data6.append(link.getText())

In [31]:
 for i in range(len(data6)):
        data6[i] = data6[i].replace("References ","")
        data6[i] = data6[i].replace("Reference ","")

se obtienen las cajas que contienen los datos de cada articulo que se extrajom de este se obtiene el numero de iteraciones para crear la lista de datos extraidos y los datos a comparar para recorrer correctamente las listas

In [32]:
iterations= []
for i in range(len(datos)):
    soup =  BeautifulSoup(datos[i])
    for link in soup.find_all('app-record',attrs={'ng-star-inserted'}):
        iterations.append(link)

In [33]:
print(len(iterations))

250


se crean los datos para almacenar los datos en el formato correcto, para esto se consideran las variaciones del formato de la fecha y de la revista, ademas se comprueba si el dato existe dentro de cada caja, si existe se copia en la lista scraoodata, en caso contrario se agrega un guion mostrando que no existe dato en esa celda

In [34]:
scrappdata = []
scrappslice = []
countdata1 = 0
countdata3 = 0
countdata2 = 0
countdata4 = 0
countdata4help = 0
countdata5 = 0
countdata5help = 0
countit = 0
for i in range(int(len(iterations))):
    if countit == 50:
        countit = 0
    scrappslice = []
    if bool(iterations[i].find_all(attrs={'data-ta':'summary-record-pubdate'})) == True and bool(iterations[i].find_all(attrs={'class':'source-info-piece ng-star-inserted'})) == False and bool(iterations[i].find_all(attrs={'class':'mat-tooltip-trigger font-size-14 borderLess-button thin-focus summary-source-title-link ng-star-inserted'})) == True:
        scrappslice.append(data[countdata1])
        if if_integer(data4[countdata4]) == False:
            dataslice = data4[countdata4].split()
            if len(dataslice) == 2:
                scrappslice.append(dataslice[1])
            else:
                scrappslice.append(dataslice[2])
            scrappslice.append(dataslice[0])    
        else:
            scrappslice.append(data4[countdata4])
            scrappslice.append("-")
        countdata4 += 1
        if bool(iterations[i].find_all(attrs={'data-ta': 'SumAuthTa-' + str(countit) + '-MainDiv-author-en'})) == True:
            scrappslice.append(data3[countdata3])
            countdata3 +=1
        else:
            scrappslice.append("-")
        scrappslice.append(data5[countdata5])
        countdata5 += 1
        if bool(iterations[i].find_all(attrs={'class':'citations ng-star-inserted'})) == True:
            scrappslice.append(data2[countdata2])
            countdata2 += 1
        else:
            scrappslice.append("-")
        scrappslice.append(data6[countdata1])           
    elif bool(iterations[i].find_all(attrs={'data-ta':'summary-record-pubdate'})) == True  and bool(iterations[i].find_all(attrs={'class':'source-info-piece ng-star-inserted'})) == False and bool(iterations[i].find_all(attrs={'class':'font-size-14 summary-source-title ng-star-inserted'})) == True:
        scrappslice.append(data[countdata1])
        if if_integer(data4[countdata4]) == False:
            dataslice = data4[countdata4].split()
            if len(dataslice) == 2:
                scrappslice.append(dataslice[1])
            else:
                scrappslice.append(dataslice[2])
            scrappslice.append(dataslice[0])    
        else:
            scrappslice.append(data4[countdata4])
            scrappslice.append("-")
        countdata4 += 1
        if bool(iterations[i].find_all(attrs={'data-ta': 'SumAuthTa-' + str(countit) + '-MainDiv-author-en'})) == True:
            scrappslice.append(data3[countdata3])
            countdata3 +=1
        else:
            scrappslice.append("-")
        scrappslice.append(helpData5[countdata5help])
        countdata5help +=1
        if bool(iterations[i].find_all(attrs={'class':'citations ng-star-inserted'})) == True:
            scrappslice.append(data2[countdata2])
            countdata2 += 1
        else:
            scrappslice.append("-")
        scrappslice.append(data6[countdata1])
    elif bool(iterations[i].find_all(attrs={'data-ta':'summary-record-pubdate'})) == False and bool(iterations[i].find_all(attrs={'class':'source-info-piece ng-star-inserted'})) == True and bool(iterations[i].find_all(attrs={'class':'mat-tooltip-trigger font-size-14 borderLess-button thin-focus summary-source-title-link ng-star-inserted'})) == True:
        scrappslice.append(data[countdata1])
        dataslice = helpData4[countdata4help].split()
        scrappslice.append(dataslice[1])
        scrappslice.append(dataslice[0])
        countdata4help += 1
        if bool(iterations[i].find_all(attrs={'data-ta': 'SumAuthTa-' + str(countit) + '-MainDiv-author-en'})) == True:
            scrappslice.append(data3[countdata3])
            countdata3 +=1
        else:
            scrappslice.append("-")
        scrappslice.append(data5[countdata5])
        countdata5 += 1
        if bool(iterations[i].find_all(attrs={'class':'citations ng-star-inserted'})) == True:
            scrappslice.append(data2[countdata2])
            countdata2 += 1
        else:
            scrappslice.append("-")
        scrappslice.append(data6[countdata1])
    elif bool(iterations[i].find_all(attrs={'data-ta':'summary-record-pubdate'})) == False and bool(iterations[i].find_all(attrs={'class':'source-info-piece ng-star-inserted'})) == True and bool(iterations[i].find_all(attrs={'class':'font-size-14 summary-source-title ng-star-inserted'})) == True:
        scrappslice.append(data[countdata1])
        dataslice = helpData4[countdata4help].split()
        scrappslice.append(dataslice[1])
        scrappslice.append(dataslice[0])
        countdata4help += 1
        if bool(iterations[i].find_all(attrs={'data-ta': 'SumAuthTa-' + str(countit) + '-MainDiv-author-en'})) == True:
            scrappslice.append(data3[countdata3])
            countdata3 +=1
        else:
            scrappslice.append("-")
        scrappslice.append(helpData5[countdata5help])
        countdata5help += 1
        if bool(iterations[i].find_all(attrs={'class':'citations ng-star-inserted'})) == True:
            scrappslice.append(data2[countdata2])
            countdata2 += 1
        else:
            scrappslice.append("-")
        scrappslice.append(data6[countdata1])
    elif bool(iterations[i].find_all(attrs={'class':'source-info-piece ng-star-inserted'})) == True and bool(iterations[i].find_all(attrs={'data-ta':'summary-record-pubdate'})) == True and bool(iterations[i].find_all(attrs={'class':'mat-tooltip-trigger font-size-14 borderLess-button thin-focus summary-source-title-link ng-star-inserted'})) == True:
        scrappslice.append(data[countdata1])
        if if_integer(data4[countdata4]) == False:
            dataslice = data4[countdata4].split()
            if len(dataslice) == 2:
                scrappslice.append(dataslice[1])
            else:
                scrappslice.append(dataslice[2])
            scrappslice.append(dataslice[0])    
        else:
            scrappslice.append(data4[countdata4])
            scrappslice.append("-")
        countdata4 += 1
        if bool(iterations[i].find_all(attrs={'data-ta': 'SumAuthTa-' + str(countit) + '-MainDiv-author-en'})) == True:
            scrappslice.append(data3[countdata3])
            countdata3 +=1
        else:
            scrappslice.append("-")
        scrappslice.append(data5[countdata5])
        countdata5 += 1
        if bool(iterations[i].find_all(attrs={'class':'citations ng-star-inserted'})) == True:
            scrappslice.append(data2[countdata2])
            countdata2 += 1
        else:
            scrappslice.append("-")
        scrappslice.append(data6[countdata1])
    elif bool(iterations[i].find_all(attrs={'class':'source-info-piece ng-star-inserted'})) == True and bool(iterations[i].find_all(attrs={'data-ta':'summary-record-pubdate'})) == True and bool(iterations[i].find_all(attrs={'class':'font-size-14 summary-source-title ng-star-inserted'})) == True: 
        scrappslice.append(data[countdata1])
        if if_integer(data4[countdata4]) == False:
            dataslice = data4[countdata4].split()
            if len(dataslice) == 2:
                scrappslice.append(dataslice[1])
            else:
                scrappslice.append(dataslice[2])
            scrappslice.append(dataslice[0])    
        else:
            scrappslice.append(data4[countdata4])
            scrappslice.append("-")
        countdata4 += 1
        if bool(iterations[i].find_all(attrs={'data-ta': 'SumAuthTa-' + str(countit) + '-MainDiv-author-en'})) == True:
            scrappslice.append(data3[countdata3])
            countdata3 +=1
        else:
            scrappslice.append("-")
        scrappslice.append(helpData5[countdata5help])
        countdata5help +=1
        if bool(iterations[i].find_all(attrs={'class':'citations ng-star-inserted'})) == True:
            scrappslice.append(data2[countdata2])
            countdata2 += 1
        else:
            scrappslice.append("-")
        scrappslice.append(data6[countdata1])

    countdata1 += 1
    scrappdata.append(scrappslice)
    countit += 1

In [35]:
print(len(scrappdata))

250


se escribe en un excel los datos recogidos y ordenados

In [46]:
import xlsxwriter
encabezados = [["Titulo","Año","Mes","Autores","Revista","Citas","referencias"]]
scrappdata = encabezados + scrappdata
workbook = xlsxwriter.Workbook('DatosWoS.xlsx')
worksheet = workbook.add_worksheet()
array = scrappdata
row = 0

for col, dataentry in enumerate(array):
    worksheet.write_row(col, row, dataentry)

workbook.close()


se obtienen los href de cada articulo que estan almacendados como una parte del link en los atributos, luego este se guarda y ejecuta recogiendo los datos de las paginas, existe un limite de aprixmadamente 500 paginas que se pueden revisar, luego la pagina bloquea la busqueda

In [37]:
hrefparts = []
for i in range(len(datos)):
    soup =  BeautifulSoup(datos[i])
    for link in soup.find_all('a', attrs={'data-ta': 'summary-record-title-link'}):
        hrefparts.append(link)


In [39]:
datos2 = []
for i in range(len(hrefparts)):
    driver.get('http://www.webofscience.com.bdigitaluss.remotexs.co' + str(hrefparts[i].attrs["href"]))
    time.sleep(2)
    d2 = driver.page_source
    datos2.append(d2)


In [40]:
print(len(datos2))

250


se genera una lista scrapBook con los datos requeridos, se guardan dentro de la lista y se preparan para guardar en un excel

In [41]:
scrapBook = []
for i in range(len(datos2)):
    Keywords_Plus_list = []
    autors = []
    Research_Areas = []
    scrapBookpart = []
    n = 0
    autorslice = ""
    Keywords_Plus_list_slice = ""
    Research_Areas_slice= ""
    soup =  BeautifulSoup(datos2[i])
    for link in soup.find_all(attrs={'cdxanalyticscategory':'wos-author_keyword_link'}):
        autors.append(link.getText())
    for link in soup.find_all(attrs={'cdxanalyticscategory':'wos-keywords-plus-link'}):
        Keywords_Plus_list.append(link.getText())
    for o in soup.find_all('span',attrs={'class':'value-wrap ng-star-inserted'}):
        n += 1
    for j in range(n):
        for link in soup.find_all('span',attrs={'data-ta':'CategoriesTa-subject-' + str(j)}):
            Research_Areas.append(link.getText())
    if bool(autors) == True:
        for m in range(len(autors)):
            if m != 0:
                autorslice += ";" +autors[m] 
            else:
                autorslice += autors[m] 
            
        scrapBookpart.append(autorslice)
    else:
        scrapBookpart.append("-")
    if bool(Keywords_Plus_list) == True:

        for m in range(len(Keywords_Plus_list)):
            if m != 0:
                Keywords_Plus_list_slice +=  ";" + Keywords_Plus_list[m] 
            else:
                Keywords_Plus_list_slice += Keywords_Plus_list[m] 
        scrapBookpart.append(Keywords_Plus_list_slice)
    else:
        scrapBookpart.append("-")
    if bool(Research_Areas) == True: 
        for m in range(len(Research_Areas)):
            if m != 0:
                Research_Areas_slice += ";" + Research_Areas[m] 
            else:        
                Research_Areas_slice += Research_Areas[m] 
        scrapBookpart.append(Research_Areas_slice) 
    else:
        scrapBookpart.append("-")
    scrapBook.append(scrapBookpart)

se escribe en un excel los datos recogidos y ordenados

In [45]:
import xlsxwriter
encabezados = [["Author Keywords","Keywords Plus","Research Areas"]]
scrapBook = encabezados + scrapBook
workbook = xlsxwriter.Workbook('DatosBookWoS.xlsx')
worksheet = workbook.add_worksheet()
array = scrapBook
row = 0

for col, dataentry in enumerate(array):
    worksheet.write_row(col, row, dataentry)

workbook.close()


se obtienen las href y se guardan en un csv por si se desea comenzar desde un archivo con los links ya extraidos, se debe de cargar el archivo y asignarlo a la lista hrefpart para poder ejecutarlod desde este punto

In [38]:
hrefs = []
for i in range(len(hrefparts)):
    hrefs.append(str(hrefparts[i].attrs["href"]))
dfhref = pd.DataFrame(hrefs,columns=["href"])
dfhref.to_csv('hrefparts.csv',index=False)

se recogen los tipos de documentos y se preparan para guardarlos en un csv para su uso en la parte 5 del taller

In [43]:
doctypes = []
doctypescon = []
for i in range(len(datos2)):
    n = 0
    doctypes_slice = []
    doctypes = []
    soup =  BeautifulSoup(datos2[i])
    for o in soup.find_all('class',attrs={'class':'value ng-star-inserte'}):
        n += 1

    for link in soup.find_all(attrs={'data-ta':'FullRTa-doctype-' + str(n)}):
        doctypes.append(link.getText())
        for m in range(len(doctypes)):
            if m != 0:
                doctypes_slice += ";" + doctypes[m] 
            else:
                doctypes_slice += doctypes[m]
        doctypescon.append(doctypes)

se escribe en un csv los datos recogidos y ordenados

In [44]:
dfdoctype = pd.DataFrame(doctypescon,columns=["tipo"])
dfdoctype.to_csv('Docktypes.csv',index=False)